## Create a template for event remapping.
This notebook works through the process analyzing the contents of the event files in
a BIDS dataset and creating a template for remapping event codes. The goal of such
a remapping is to orthogonalize event design and make the meanings of contents
of the event file more clear.

Suppose we want to remap each unique combination in the key columns [`value`, `trial_type`]
into a new combination in the target columns [`event_type`, `task_role`, `shift_cond`].

**Table 1:** Sample template created from a BIDS dataset.

| value | trial_type | event_type | task_role | shift_cond |
| ----- | ---------- | ---------- | --------- | ---------- |
| 11    | 1          | n/a        | n/a       | n/a        |
| 31    | 1          | n/a        | n/a       | n/a        |
| 41    | 1          | n/a        | n/a       | n/a        |
| 42    | 2          | n/a        | n/a       | n/a        |
| 201   | 3          | n/a        | n/a       | n/a        |

After creating a template file for your data, you must manually edit it
to fill in the desired mapping.  Table 2 gives an example of a potential
mapping.

**Table 2:** An excerpt of the completed template for excerpt of Table 1.

| value | trial_type | event_type   | task_role         | shift_cond    |
| ----- | ---------- | ------------ | ----------------- | ------------- |
| 11    | 1          | hear_cue     | cue               | auditory_cond |
| 31    | 1          | high_tone    | target_attended   | auditory_cond |
| 41    | 1          | light_bar    | target_unattended | auditory_cond |
| 42    | 2          | light_bar    | target_attended   | visual_cond   |
| 201   | 3          | button_press | target_detected   | shift_cond    |


#### Set dataset location

The example used in this notebook is reduced version of an auditory attention shift
dataset which is available at
[https://github.com/hed-standard/hed-examples/data/eeg_ds0028932](https://github.com/hed-standard/hed-examples/data/eeg_ds0028932).

To run this notebook, you will need download this dataset and set the `bids_root_path`
variable to the local path of the dataset's root directory.

Alternatively, you can set `bids_root_path` to the full path of your own BIDS dataset.

In [1]:
bids_root_path = "D:/eeg_ds002893s"

#### Set key and target columns

The goal is to create a mapping of each unique combination of the key columns
to a combination of target column values.

(You will need to set the actual target column values manually after a template
file is created.)

In [2]:
key_columns = ["value", "trial_type"]
target_columns = ["event_type", "task_role", "shift_cond"]

#### Get a list of event files

The next example recursively traverses the directory tree and produces
a list of the full paths of the dataset event files.

Event files have extension `.tsv` and the file names end with `_events`.
You may wish to check the returned list to verify that the expected event files
are in the dataset.

In [3]:
from hed.tools import get_file_list
event_file_list = get_file_list(bids_root_path, types=[".tsv"], suffix="_events")
print(f"Bids dataset {bids_root_path} has {len(event_file_list)} event files")

Bids dataset D:/eeg_ds002893s has 6 event files


#### Find unique key column combinations

Use the `KeyTemplate` class to find the unique value combinations of a list of columns.

If you just want a summary of the unique values in individual columns of all
event files, use the `ColumnDict` class as illustrated in notebook
`summarize_events.ipynb`.


In [4]:
from hed.tools import KeyTemplate
template = KeyTemplate(key_columns)
for file in event_file_list:
    template.update(file)
template.resort()
template.print()

Counts for key [['value', 'trial_type']]:
[11, 1]	228
[12, 1]	228
[13, 1]	456
[14, 1]	456
[15, 1]	1822
[16, 1]	1820
[21, 2]	252
[22, 2]	252
[23, 2]	505
[24, 2]	504
[25, 2]	2010
[26, 2]	2012
[28, 2]	2
[31, 3]	720
[32, 3]	719
[37, 3]	960
[38, 3]	960
[39, 3]	480
[202, 1]	13
[202, 2]	17
[202, 3]	42
[212, 2]	3
[310, 3]	480
[311, 3]	3838
[312, 3]	3832
[313, 3]	1920
[314, 3]	1915
[1201, 1]	433
[2201, 2]	502
[3201, 3]	1846


#### Save the template for editing

In [5]:
import os
df = template.make_template(additional_cols=target_columns)
template_file = os.path.join(bids_root_path, "remap_event_template.tsv")
df.to_csv(template_file, sep='\t', index=False)

#### Manually edit template

To do remapping, you will need to edit the `event_template.tsv` spreadsheet
created in the previous step to indicate how each combination of [`value`, `trial_type`]
is mapped into a combination of [`event_type`, `task_role`, `shift_cond`].

Once you have filled in the mapping, you can use the `remap_events.ipynb` notebook
to remap all the event files in your BIDS dataset.